In [ ]:
from ppsim import species, Simulation, Specie
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
triplets = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
            [0, 3, 6], [1, 4, 7], [2, 5, 8],
            [0, 4, 8], [2, 4, 6]] # winning combinations

In [ ]:
def gen_string(prefix: str, n: int) -> str:
    to_return = ""
    for i in range(n):
        to_return += prefix + str(i) + " "
    return to_return[:-1]

In [ ]:
def board(sim: Simulation) -> str:
    # return board state
    states = np.zeros((3,3), dtype=str)
    for i in range(9):
        if Specie(name=f'o{i}') in sim.config_dict.keys():
            states[i // 3, i % 3] = 'o'
        elif Specie(name=f'x{i}') in sim.config_dict.keys():
            states[i // 3, i % 3] = 'x'
        else:
            states[i // 3, i % 3] = '_'
    return states

In [ ]:
def game_state(sim: Simulation) -> str:
    # return game state
    if Specie(name='win') in sim.config_dict.keys():
        return 'crn wins'
    elif Specie(name='draw') in sim.config_dict.keys():
        return 'draw'
    elif Specie(name='lose') in sim.config_dict.keys():
        return 'crn loses'
    else:
        return 'playing'

In [170]:
c, h, w = species("c h w") # set up move markers (c, h), waste molecule (w)
win, lose, draw = species("win lose draw") # set up game result markers
checkc, checkh, check_fuel = species("checkc checkh check_fuel") # checking for stopping condition, resuming game
continue_gameh, continue_gamec, converter = species("continue_gameh continue_gamec converter") # continue game
# toggle clean up on, clean up off
cuon, cuoff = species("cuon cuoff")
# option, try squares (this computes probabilities)
opt = species(gen_string("opt", 9))
t = species(gen_string("t", 9))
# squares indicators - must be only one of these each
b = species(gen_string("b", 9))
x = species(gen_string("x", 9))
o = species(gen_string("o", 9))
# square interactors - can be many of these but only use in non-changing phases
b_temp = species(gen_string("b_temp", 9))
x_temp = species(gen_string("x_temp", 9))
o_temp = species(gen_string("o_temp", 9))
# intermediates for computation
filled = species(gen_string("filled", 8))
inter_triplet = species(gen_string("inter_triplet", 24))
inter_sum = species(gen_string("inter_sum", 8))
continue_triplet = species(gen_string("continue_triplet", 8))
# track move that was made
m = species(gen_string("m", 9))
# inject a move: human's turn, produces mh to make human move
inject = species(gen_string("i", 9))
mv = species(gen_string("mv", 9)) # checks if move is valid
mh = species(gen_string("mh", 9)) # makes move
mm = species(gen_string("mx", 9)) # move has been made

In [174]:
rules = []
# square molecule reactions
for i in range(9):
    # generate random move from blank squares: c + b[i] -> c + b[i] + t[i]
    rules.append(w + c >> c + opt[i])
    rules.append(opt[i] + b[i] >> t[i] + b[i])

    # make_move: requires a blank square and the move marker -> fills square and records move
    rules.append(t[i] + c >> o[i] + m[i])
    # move recorded -> decompose blank, generate human's turn
    rules.append(m[i] + b[i] >> w + checkc)

    # if human's turn, do the same thing, generate checking phase
    rules.append(inject[i] + b[i] >> mv[i] + b[i]) # move is valid
    rules.append(mv[i] + h >> mh[i] + h) # move is valid and human's turn
    rules.append(mh[i] + b[i] >> mm[i] + x[i]) # make move
    rules.append(mm[i] + h >> w + checkh) # end human's turn, move to checking phase

    # clean up step
    rules.append(cuon + mh[i] >> cuon + w)
    rules.append(cuon + mm[i] >> cuon + w)
    rules.append(cuon + mv[i] >> cuon + w)
    rules.append(cuon + opt[i] >> cuon + w)
    rules.append(cuon + t[i] >> cuon + w)
    rules.append(cuon + b_temp[i] >> cuon + w)
    rules.append(cuon + x_temp[i] >> cuon + w)
    rules.append(cuon + o_temp[i] >> cuon + w)

    # generate temporaries - for checking but also all the time
    rules.append(check_fuel + x[i] >> x[i] + x_temp[i])
    rules.append(check_fuel + o[i] >> o[i] + o_temp[i])
    rules.append(check_fuel + b[i] >> b[i] + b_temp[i])

# control molecule clean up
rules.append(cuon + check_fuel >> cuon + w) # clean up check fuel
rules.append(cuon + continue_gameh >> cuon + w) # clean up continue game
rules.append(cuon + continue_gamec >> cuon + w) # clean up continue game
rules.append(cuon + converter >> cuon + w) # clean up converter

# control molecule reactions
rules.append(cuoff + cuon >> w + w) # toggle off clean up
rules.append(continue_gameh + converter >> continue_gamec + w) # continue game after computer's turn
rules.append(checkh + continue_gameh >> c + w) # toggle off checking phase after human's turn
rules.append(checkc + continue_gamec >> h + w) # toggle off checking phase after computer's turn

# triplet reactions - checking phase
for i, triplet in enumerate(triplets):
    # generate win if all three are O: o0 + o1 + o2 -> win
    rules.append(o_temp[triplet[0]] + o_temp[triplet[1]] >> inter_triplet[i] + w)
    rules.append(inter_triplet[i] + o_temp[triplet[2]] >> win + w)
    # generate lose if all three are X: x0 + x1 + x2 -> lose
    rules.append(x_temp[triplet[0]] + x_temp[triplet[1]] >> inter_triplet[i+8] + w)
    rules.append(inter_triplet[i+8] + x_temp[triplet[2]] >> lose + w)
    # generate filled[i] if all three are filled: b0 + b1 + b2 -> filled[i]
    rules.append(b_temp[triplet[0]] + b_temp[triplet[1]] >> inter_triplet[i+16] + w)
    rules.append(inter_triplet[i+16] + b_temp[triplet[2]] >> filled[i] + w)
    # generate continue_triplet[i] if none of the above (this is the hard one)
    rules.append(b_temp[triplet[0]] >> continue_triplet[i])
    rules.append(b_temp[triplet[1]] >> continue_triplet[i])
    rules.append(b_temp[triplet[2]] >> continue_triplet[i]) # if any blanks, continue
    rules.append(x_temp[triplet[0]] + o_temp[triplet[1]] >> continue_triplet[i] + w) # xo_
    rules.append(o_temp[triplet[0]] + x_temp[triplet[1]] >> continue_triplet[i] + w) # ox_
    rules.append(x_temp[triplet[1]] + o_temp[triplet[2]] >> continue_triplet[i] + w) # _xo
    rules.append(o_temp[triplet[1]] + x_temp[triplet[2]] >> continue_triplet[i] + w) # _ox
    rules.append(x_temp[triplet[0]] + o_temp[triplet[2]] >> continue_triplet[i] + w) # x_o
    rules.append(o_temp[triplet[0]] + x_temp[triplet[2]] >> continue_triplet[i] + w) # o_x

    # clean up triplets
    rules.append(cuon + inter_triplet[i] >> cuon + w)
    rules.append(cuon + inter_triplet[i+8] >> cuon + w)
    rules.append(cuon + inter_triplet[i+16] >> cuon + w)
    rules.append(cuon + continue_triplet[i] >> cuon + w)
    rules.append(cuon + inter_sum[i] >> cuon + w)

# summing triplet results: draw
rules.append(filled[0] + filled[1] >> inter_sum[0] + w)
rules.append(inter_sum[0] + filled[2] >> draw + w)
rules.append(filled[3] + filled[4] >> inter_sum[1] + w)
rules.append(inter_sum[1] + filled[5] >> draw + w)

# summing triplet results: continue only if every triplet continues
rules.append(continue_triplet[0] + continue_triplet[1] >> inter_sum[2] + w)
rules.append(continue_triplet[2] + continue_triplet[3] >> inter_sum[3] + w)
rules.append(continue_triplet[4] + continue_triplet[5] >> inter_sum[4] + w)
rules.append(continue_triplet[6] + continue_triplet[7] >> inter_sum[5] + w) # level one
rules.append(inter_sum[2] + inter_sum[3] >> inter_sum[6] + w)
rules.append(inter_sum[4] + inter_sum[5] >> inter_sum[7] + w) # level two
rules.append(inter_sum[6] + inter_sum[7] >> continue_gameh + w) # level three


In [164]:
init_config = {}
init_config[w] = 100 # surplus waste - I think this actually ends up affecting the probabilities
init_config[c] = 0 # crn's move
init_config[h] = 1 # not human's move
init_config[cuon] = 0 # add to clean up
init_config[cuoff] = 0 # add to stop cleaning up
move_probabilities = [0, 0, 0, 0, 0, 0, 0, 0, 0] # t values
for i in range(9):
    init_config[b[i]] = 1 # all squares set to blank
    init_config[x[i]] = 0
    init_config[o[i]] = 0
    init_config[m[i]] = 0
    init_config[mh[i]] = 0
    init_config[inject[i]] = 0
    init_config[opt[i]] = 0
    init_config[t[i]] = move_probabilities[i]

In [172]:
# -1 for cuon, -2 for cuoff, -3 inject check fuel, -4 inject converter, 0 through 8 for corresponding move
def injection(n: int, init_config: dict) -> dict:
    if n == -1:
        init_config[cuon] = 1
    elif n == -2:
        init_config[cuoff] = 1
    elif n == -3:
        init_config[check_fuel] = 400
    elif n == -4:
        init_config[converter] = 1
    else:
        init_config[inject[n]] = 1
    return init_config

In [177]:
def play_turn(n: int, config: dict) -> Simulation:
    sim = Simulation(injection(-1, config), rules) # clean up on
    sim.run()
    sim = Simulation(injection(-2, sim.config_dict), rules) # clean up off
    sim.run()
    sim = Simulation(injection(n, sim.config_dict), rules) # inject human move
    sim.run()
    sim = Simulation(injection(-3, sim.config_dict), rules) # inject fuel for checking human move
    sim.run()
    print(board(sim))
    sim = Simulation(injection(-1, sim.config_dict), rules) # clean up on
    sim.run()
    sim = Simulation(injection(-2, sim.config_dict), rules) # clean up off
    sim.run()
    sim = Simulation(injection(-3, sim.config_dict), rules) # inject fuel for checking crn move
    sim.run()
    sim = Simulation(injection(-4, sim.config_dict), rules) # inject converter to advance game
    sim.run()
    print(game_state(sim))
    return sim

In [186]:
# first move
sim = play_turn(4, init_config)

[['_' 'o' '_']
 ['_' 'x' '_']
 ['_' '_' '_']]


playing


In [190]:
# next moves
sim = play_turn(3, sim.config_dict)

[['o' 'o' 'o']
 ['_' 'x' 'x']
 ['o' 'x' 'x']]


crn wins


In [ ]:
# plot results
sim.history[['c','h','w']].plot()
plt.title("play or don't play")
plt.xlim(0, sim.times[-1])
plt.show()